In [4]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/cambrian_reco/'
from hydra import compose, initialize
import os
import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/cambrian_reco/')

import xml.etree.ElementTree as ET

tree = ET.parse('../conf/application.run.xml')
root = tree.getroot()
envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import pandas as pd
import numpy as np

from sql.PostgresDatabaseOperation import PostgresDatabaseOperation
from sql.ArticleSelectionSQL import ArticleSelectionSQL

In [5]:
from src._utils import get_time_decay_article_priors_for_all_interacted_articles

In [21]:
def get_day_wise_article_a_b() -> pd.DataFrame:
    # TODO: - to evaluate if we need to take count distinct for feed id in case articles can duplicate
    # TODO: - do this only for candidate articles
    with PostgresDatabaseOperation() as cursor:
        interaction_sql = f"""SELECT article_id, CAST(created_at AS DATE) as feed_date, SUM(CAST(is_article_opened AS INT)) as a, COUNT(activity_id) as b
                            FROM user_article_interactions
                            GROUP BY article_id, feed_date 
                            """
        cursor.execute(interaction_sql)
        results = cursor.fetchall()
        df = pd.DataFrame(results, columns=['article_id', 'feed_date', 'a', 'b'])
        return df

In [6]:
df = get_time_decay_article_priors_for_all_interacted_articles()

In [8]:
df1 = ArticleSelectionSQL.get_day_wise_article_a_b()

In [118]:
df1['feed_date'] = pd.to_datetime(df1['feed_date'])

In [122]:
df1['days'] = (datetime.today() - df1['feed_date']).dt.days

In [129]:
%timeit df1['decay'] = alternative_decay_faster(df1['days'])

99.8 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [134]:
(datetime(2023,10,16,23,0,0) - df1['feed_date']).dt.days

0        0
1        0
2      -24
3        0
4      -30
        ..
2689    -5
2690    -7
2691     0
2692     4
2693    -7
Name: feed_date, Length: 2694, dtype: int64

In [136]:
df1['a'] = df1['a'].apply(lambda x: 0 if x == 0 else None)

In [142]:
%timeit df1['a'] = df1['a'].fillna(1)

29.3 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [143]:
df1.fillna({'feed_a': 0, 'feed_b': 0, 'search_a': 0, 'search_b': 0, 'prior_a': 1, 'prior_b': 10})

,article_id,feed_date,a,b,days_ago,decay,days
0,651dd0cea662d76276b7d32b,2023-10-16,0.0,1,-52,0.525915,51
1,651dcfd1a662d76276b7d062,2023-10-16,0.0,1,-52,0.525915,51
2,652fcb6e1e5cc42b1b13a172,2023-11-09,0.0,1,-28,0.620851,27
3,651de34aa662d76276b803fa,2023-10-16,0.0,1,-52,0.525915,51
4,652a045b50af0e25a9122fd2,2023-11-15,0.0,1,-22,0.661951,21
...,...,...,...,...,...,...,...
2689,6530d42c1e5cc42b1b13a71c,2023-10-21,0.0,1,-47,0.540000,46
2690,653168441e5cc42b1b13a899,2023-10-23,0.0,1,-45,0.546250,44
2691,652cace850af0e25a91230be,2023-10-16,0.0,1,-52,0.525915,51
2692,651dce28a662d76276b7cbb3,2023-10-12,0.0,1,-56,0.516000,55


In [139]:
df1.isnull().sum()

article_id     0
feed_date      0
a             81
b              0
days_ago       0
decay          0
days           0
dtype: int64

In [137]:
df1

,article_id,feed_date,a,b,days_ago,decay,days
0,651dd0cea662d76276b7d32b,2023-10-16,0.0,1,-52,0.525915,51
1,651dcfd1a662d76276b7d062,2023-10-16,0.0,1,-52,0.525915,51
2,652fcb6e1e5cc42b1b13a172,2023-11-09,0.0,1,-28,0.620851,27
3,651de34aa662d76276b803fa,2023-10-16,0.0,1,-52,0.525915,51
4,652a045b50af0e25a9122fd2,2023-11-15,0.0,1,-22,0.661951,21
...,...,...,...,...,...,...,...
2689,6530d42c1e5cc42b1b13a71c,2023-10-21,0.0,1,-47,0.540000,46
2690,653168441e5cc42b1b13a899,2023-10-23,0.0,1,-45,0.546250,44
2691,652cace850af0e25a91230be,2023-10-16,0.0,1,-52,0.525915,51
2692,651dce28a662d76276b7cbb3,2023-10-12,0.0,1,-56,0.516000,55


In [123]:
df1

,article_id,feed_date,a,b,days_ago,decay,days
0,651dd0cea662d76276b7d32b,2023-10-16,0,1,-52,1.0,51
1,651dcfd1a662d76276b7d062,2023-10-16,0,1,-52,1.0,51
2,652fcb6e1e5cc42b1b13a172,2023-11-09,0,1,-28,1.0,27
3,651de34aa662d76276b803fa,2023-10-16,0,1,-52,1.0,51
4,652a045b50af0e25a9122fd2,2023-11-15,0,1,-22,1.0,21
...,...,...,...,...,...,...,...
2689,6530d42c1e5cc42b1b13a71c,2023-10-21,0,1,-47,1.0,46
2690,653168441e5cc42b1b13a899,2023-10-23,0,1,-45,1.0,44
2691,652cace850af0e25a91230be,2023-10-16,0,1,-52,1.0,51
2692,651dce28a662d76276b7cbb3,2023-10-12,0,1,-56,1.0,55


In [11]:
df1.a.sum()

81

In [12]:
df1.b.sum()

2776

In [13]:
from src._utils import apply_article_pref_decay_by_feed_date

In [14]:
df2 = apply_article_pref_decay_by_feed_date(df1)

In [1]:
import numpy as np
import pandas as pd

In [2]:
num_hours = lambda days: days * 24 

In [17]:
6/24

0.25

In [122]:
def timeliness_weight(hours_since_publication, page_number=1):
    # https://www.desmos.com/calculator/qku1api5x6
    return 0.25 + (0.75/(1+np.exp((np.log2(hours_since_publication) - 7)/(page_number))))

In [100]:
def alternative_decay(days):
    return 0.33 + (0.66/(1 + (days*np.exp(-3))))

In [125]:
def alternative_decay_faster(days):
    return 0.34 + (0.66/(1 + 0.05*days))

In [105]:
np.exp(-3)

0.049787068367863944

In [114]:
alternative_decay1(4)

0.8900000000000001

In [115]:
from datetime import datetime

In [116]:
datetime.today() - 

datetime.datetime(2023, 12, 6, 17, 48, 4, 540788)

In [107]:
%timeit alternative_decay1(3)

51.7 ns ± 0.963 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [103]:
%timeit alternative_decay(3)

607 ns ± 6.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [104]:
%timeit interaction_decay(3)

785 ns ± 5.17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [90]:
def interaction_decay(days):
    # https://www.desmos.com/calculator/qku1api5x6
    # return 1/days
    return 0.33 + (0.66/(1+np.exp(np.log(days + 0.00001)-3)))

In [101]:
alternative_decay(30)

0.5946762954669782

In [93]:
interaction_decay(30)

0.5946762426221333

In [124]:
for day in [0.25, 1, 3, 7, 14, 30, 90, 360, 720]:
    weight = timeliness_weight(num_hours(day))
    print(f'hours for {day} is {num_hours(day)} and weight of {weight}')

hours for 0.25 is 6.0 and weight of 0.9910378141300826
hours for 1 is 24 and weight of 0.9384751179747243
hours for 3 is 72 and weight of 0.7722780907307086
hours for 7 is 168 and weight of 0.5523696647847406
hours for 14 is 336 and weight of 0.3992785384042919
hours for 30 is 720 and weight of 0.3073234640573269
hours for 90 is 2160 and weight of 0.26250887579723964
hours for 360 is 8640 and weight of 0.2517176632049459
hours for 720 is 17280 and weight of 0.2506328090936522


In [135]:
def softmax(arr, t):
    exp_arr = [np.exp(x/t) for x in arr]
    exp_arr_sum = np.sum(exp_arr)
    return [x/exp_arr_sum for x in exp_arr]

In [136]:
cluster_prop = np.random.rand(30)

In [140]:
cluster_prop

array([0.74368719, 0.75260124, 0.53138819, 0.162973  , 0.96440304,
       0.16611312, 0.48678248, 0.98910907, 0.27297829, 0.9024516 ,
       0.0486279 , 0.39512626, 0.00386668, 0.67022996, 0.42663124,
       0.88509956, 0.9089922 , 0.66778674, 0.28804467, 0.20347509,
       0.67435651, 0.12230065, 0.35860683, 0.02847255, 0.88014925,
       0.19427015, 0.13613699, 0.32026216, 0.13993498, 0.91130383])

In [144]:
softmax(cluster_prop, 0.01)

[2.0222329446744063e-11,
 4.931326728805649e-11,
 1.2184361022727835e-20,
 1.2182450229620534e-36,
 0.07787517608740142,
 1.66766239171267e-36,
 1.4079966222560547e-22,
 0.9212305994803663,
 7.297992526684377e-32,
 0.00015881147063309738,
 1.3176195769775564e-41,
 1.4723893939626566e-26,
 1.4991150490553414e-43,
 1.305041403672163e-14,
 3.4376867297983563e-25,
 2.8008671497454507e-05,
 0.000305447600401855,
 1.0221547942053135e-14,
 3.2925178027394974e-31,
 6.993873571941243e-35,
 1.9716876616167863e-14,
 2.086203414591155e-38,
 3.819477406900956e-28,
 1.7557153202622058e-42,
 1.7072741356198383e-05,
 2.7858176188087203e-35,
 8.322643443823771e-38,
 8.25497904955951e-30,
 1.2167625346002248e-37,
 0.00038488387876516164]

In [ ]:
softmax: lambda arr: [np.exp(x)]

In [16]:
np.log2(num_hours(1))

4.584962500721156

In [ ]:
4, 8, 16, 32

In [120]:
timeliness_weight(24,)

0.7682842608550897

In [21]:
%timeit timeliness_weight(168)

939 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
